In [1]:
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from fisher_resnet import *

Specify number of run and dataset version

In [1]:
run_number = 0
data = 'cifar10'

Experiments were performed for run_number = 0, 1, 2 and data="cifar10", "cifar100"

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


Learning rate scheduler

In [4]:
class LinearLR(torch.optim.lr_scheduler._LRScheduler):

    def __init__(self, optimizer, num_epochs, last_epoch=-1):
        self.num_epochs = max(num_epochs, 1)
        super(LinearLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        res = []
        for lr in self.base_lrs:
            res.append(np.maximum(
                lr * np.minimum(-(self.last_epoch + 1) * 1. / self.num_epochs + 1., 1.), 0.))
        return res

Erase network

In [5]:
def erase_net_tmp(net):
    for child1 in net.children():
        if isinstance(child1, nn.Sequential):
            for child2 in child1.children():
                if isinstance(child2, PreActBottleneck):
                    for child3 in child2.children():
                        if isinstance(child3, MaskedConv2d) or isinstance(child3, MaskedLinear):
                            child3.erase_tmp()

Specify number of classes

In [6]:
if data == 'cifar10':
    n_classes = 10
    base_path = 'resnet_cifar10/'
    path = base_path + '0_' + str(run_number) + '.pth'
else:
    n_classes = 100
    base_path = 'resnet_cifar100/'
    path = base_path + '0_' + str(run_number) + '.pth'

Make network instance, move it to gpu's

In [6]:
net = resnet50(n_classes)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)

net.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): PreActBottleneck(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): MaskedConv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): MaskedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): MaskedConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (shortcut): Sequential(
        (0): MaskedConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
    )
    (1): PreActBottleneck(
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True

Load data, make loaders: trainloader with train data for training, testloader with test data for testing, trainloader_prun with batch of train data for pruning (identify the sensitivity of connections).

In [7]:
transform_train = transforms.Compose([
    transforms.Pad(4, padding_mode='symmetric'),
    transforms.RandomCrop(32),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [8]:
if data == 'cifar10':
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
else:
    trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform_train)
    testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform_test)    

Files already downloaded and verified
Files already downloaded and verified


In [9]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True)

In [10]:
trainloader_prun = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True)

In [11]:
testloader = torch.utils.data.DataLoader(testset, batch_size=1024, shuffle=False)

Define function for one epoch training and testing

In [13]:
def train(net, optimizer):
    train_loss = 0.
    net.train()
    for input, target in tqdm(trainloader):
        input = input.to(device)
        target = target.to(device)
        
        optimizer.zero_grad()
        output = net(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
    print ('Train loss ', train_loss/len(trainloader))
    
def test(net):
    net.eval()
    with torch.no_grad():
        test_loss = 0.
        correct = 0.
        for input, target in tqdm(testloader):
            input = input.to(device)
            target = target.to(device)
            
            output = net(input)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            _, predicted = torch.max(output.data, 1)
            correct += (predicted == target).sum().item()
            
        test_loss /= len(testset)

        print ('Test loss ', test_loss)
        print ('Test accuracy ', correct / len(testset))

Define loss function, optimizer and learning rate scheduler

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam([p for p in net.parameters() if p.requires_grad], lr=1e-3)
n_epochs = 200
lr_scheduler = LinearLR(optimizer, n_epochs)

Training cycle: 50 epochs

In [15]:
for epoch in range(n_epochs):
    if epoch > 50:
        break
    print ('Epoch ', epoch)
    lr_scheduler.step()
    train(net, optimizer)
    test(net)

  0%|          | 0/196 [00:00<?, ?it/s]

Epoch  0


  0%|          | 0/10 [00:00<?, ?it/s]

Train loss  1.7083885791350384


  0%|          | 0/196 [00:00<?, ?it/s]

...

Test loss  0.34511240234375
Test accuracy  0.9221
Epoch  50


  0%|          | 0/10 [00:00<?, ?it/s]

Train loss  0.048123802210451386


100%|██████████| 10/10 [00:06<00:00,  1.54it/s]

Test loss  0.3576927993774414
Test accuracy  0.9186


Save the model to directory

In [17]:
torch.save(net.state_dict(), path)

Function for network pruning

In [15]:
def prune_net(net, percentile, N=128):
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), 
                       lr=5e-4)
    erase_net_tmp(net)

    inc = 0
    for input, target in trainloader_prun:
        if inc > N - 1: break
        input, target = input.cuda(), target.cuda()

        optimizer.zero_grad()
        output = net(input)
        loss = criterion(output, target)
        loss.backward()

        for child1 in net.children():
            if isinstance(child1, nn.Sequential):
                for child2 in child1.children():
                    if isinstance(child2, PreActBottleneck):
                        for child3 in child2.children():
                            if isinstance(child3, MaskedConv2d) or isinstance(child3, MaskedLinear):
                                child3.weight_tmp = (child3.weight_tmp + 
                                child3.weight.data ** 2 * child3.weight.grad.data ** 2 / len(trainloader_prun) / 2.)
                                
                                if child3.bias is not None:
                                    child3.bias_tmp = (child3.bias_tmp +
                                    child3.bias.data ** 2 * child3.bias.grad.data ** 2 / len(trainloader_prun) / 2.)

        inc += 1
        
    values = []
    for child1 in net.children():
        if isinstance(child1, nn.Sequential):
            for child2 in child1.children():
                if isinstance(child2, PreActBottleneck):
                    for child3 in child2.children():
                        if isinstance(child3, MaskedConv2d) or isinstance(child3, MaskedLinear):
                            
                            values += [child3.weight_tmp.view(-1)]
                            
                            if child3.bias is not None:
                                values += [child3.bias_tmp.view(-1)]
                
    values = torch.cat(values, dim=0).cpu().detach().numpy()
    value = np.percentile(values, percentile)
    
    for child1 in net.children():
        if isinstance(child1, nn.Sequential):
            for child2 in child1.children():
                if isinstance(child2, PreActBottleneck):
                    for child3 in child2.children():
                        if isinstance(child3, MaskedConv2d) or isinstance(child3, MaskedLinear):
                            child3.truncate(value)
    return net

Define percentiles

In [16]:
percentiles = np.arange(0, 95, 15)[1:]
percentiles

array([15, 30, 45, 60, 75, 90])

Pruning retraining cycle: prune for each percentile, retrain to recover initial quality

In [17]:
for i, percentile in enumerate(percentiles):
    
    net = resnet50(n_classes)
    net.load_state_dict(torch.load(path))
    net.to(device)
    
    net = prune_net(net, percentile)
        
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), 
                      lr=5e-4)

    n_epochs = 1
    for epoch in range(n_epochs):
        print ('Epoch ', epoch)
        test(net)
        train(net, optimizer)
    
    test(net)
    
    torch.save(net.state_dict(), base_path + str(i+1) + '_' + str(run_number) + '.pth')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch  0


  0%|          | 0/196 [00:00<?, ?it/s]

Test loss  317.1786578125
Test accuracy  0.7911


  0%|          | 0/10 [00:00<?, ?it/s]

...

Test loss  12.77069228515625
Test accuracy  0.3144


  0%|          | 0/10 [00:00<?, ?it/s]

Train loss  0.03013655732442834


100%|██████████| 10/10 [00:06<00:00,  1.54it/s]


Test loss  0.4074203857421875
Test accuracy  0.9299
